In [1]:
%config InlineBackend.figure_format = "svg"


In [2]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import cm
from matplotlib.collections import LineCollection


In [3]:
import sys
import pathlib
import os
sys.path.append(str(pathlib.Path(os.path.abspath('')).resolve().parent.parent))
from IPython.display import display, HTML

from _9_Mechanics.Spiral import Spiral
from _9_Mechanics.Tribology import Tribology

TB = Tribology()


In [4]:
R = 6.35 / 2
Rg = R * 1.08
dR = R * 0.08 / np.sqrt(2)
PCR = 20.875

SP1 = Spiral(alp=np.deg2rad(1e-1), l=10.0, r=PCR, eta=np.array([[-dR, -dR],[-dR, dR]]), R=np.array([Rg, Rg]))
SP2 = Spiral(alp=np.deg2rad(-1e-1), l=10.0, r=PCR, eta=np.array([[dR, dR],[dR, -dR]]), R=np.array([Rg, Rg]))

x_ball = np.array([0.0, PCR, 0.0])
rho = 7.9e-3 # [g/mm^3]
Ball_m = 4/3 * np.pi * R**3 * rho # [g]
Ball_r_inv = 1.0 / R
v_ball = np.zeros(3)
w_ball = np.zeros(3)
zeta = 0.3
xvw_ball = np.concatenate([x_ball, v_ball, w_ball])
E_Reduced = 231e3 # [N/mm^2]


In [12]:
p0 = np.array([4,0,0])
x0 = np.array([2,0,0])
v0 = np.array([0,0,100])
w0 = np.array([0,0,5])

TB.surface_velocity(x0, p0, v0, w0)

for i in range(2):
    print(i)


0
1


In [6]:
def vaT_ball(xvw_ball, t, R, SP1, SP2, Ball_r_inv, zeta, E_Reduced):
    x_ball = xvw_ball[0:3]
    v_ball = xvw_ball[3:6]
    w_ball = xvw_ball[6:9]
    
    cos_sin1, dx1, exyz1 = SP1.get_contact(x_ball, R, 0)
    SP1_r_inv = SP1.get_rho(cos_sin1[0], 0)
    R_Reduced1 = 1.0 / (Ball_r_inv + SP1_r_inv)
    k1, a1, b1 = TB.BrewHamrock(R_Reduced1[0], R_Reduced1[1], dx1, E_Reduced)
    v1 = v_ball.dot(exyz1)
    F1 = - TB.Tsuji(k1, zeta, Ball_m, v1, dx1) * exyz1

    cos_sin2, dx2, exyz2 = SP2.get_contact(x_ball, R, 0)
    SP2_r_inv = SP2.get_rho(cos_sin2[0], 0)
    R_Reduced2 = 1.0 / (Ball_r_inv + SP2_r_inv)
    k2, a2, b2 = TB.BrewHamrock(R_Reduced2[0], R_Reduced2[1], dx2, E_Reduced)
    v2 = v_ball.dot(exyz2)
    F2 = - TB.Tsuji(k2, zeta, Ball_m, v2, dx2) * exyz2

    a_ball = (F1 + F2) / Ball_m
    
    return np.concatenate([v_ball, a_ball])


In [7]:
from scipy.integrate import odeint
t = np.linspace(0, 0.2, 1000)
sol = odeint(vaT_ball, xvw_ball, t, args=(R, SP1, SP2, Ball_r_inv, zeta, E_Reduced))

fig, ax = plt.subplots()
ax.plot(t, sol[:,1])

plt.xlabel('Time: T [s]')
plt.ylabel('y [mm]')


RuntimeError: The size of the array returned by func (6) does not match the size of y0 (9).